In [19]:
import arcpy
import pandas as pd
import psycopg2
from psycopg2 import sql
import numpy as np
import os
import folium

In [40]:
cwd = os.getcwd() # This is a global variable for where the notebook is (must change if running in arcpro)

# Make it workspace

arcpy.env.workspace = os.path.join(cwd, '..', '..', 'data', 'QAQC.gdb')

arcpy.env.overwriteOutput = True # Overwrite layers is okay

save_path = os.path.join(arcpy.env.workspace, "help")

In [5]:
connection = psycopg2.connect(host = '34.27.240.213',
                              database = 'lab0',
                              user = 'postgres',
                              password = 'Homework13',
                              port = '5432')
connection.closed

0

In [6]:
cur = connection.cursor()

In [51]:
cur.close()
connection.close()

In [7]:
sql_query_rename = "ALTER TABLE lab0.public.purpleair_historic RENAME COLUMN sensor_index TO sensor_index1;"
cur.execute(sql_query_rename)

sql_query = """
    SELECT *
    FROM lab0.public.purpleair_historic
    INNER JOIN lab0.public.purpleair_stations ON lab0.public.purpleair_historic.sensor_index1 = lab0.public.purpleair_stations.sensor_index
"""

# Execute the query
cur.execute(sql_query)
# Fetch the results as a pandas dataframe
join_df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

In [8]:
historic_latlong = join_df.drop(['sensor_index','timestamp', 'last_modified', 'date_created', 'last_seen', 'name', 'location_type', 'firmware_version', 'uptime', 'position_rating', 'altitude', 'channel_state', 'channel_flags'], axis=1)

In [12]:
historic_latlong.head()

,sensor_index1,humidity,temperature,pressure,pm2_5,latitude,longitude,wkt
0,3088,40.201,17.581,991.743,0.6330,44.935818,-93.21752,0101000020E6100000758E01D9EB4D57C00A815CE2C877...
1,3088,42.605,0.743,987.762,0.0785,44.935818,-93.21752,0101000020E6100000758E01D9EB4D57C00A815CE2C877...
2,3088,48.795,29.080,990.701,54.4190,44.935818,-93.21752,0101000020E6100000758E01D9EB4D57C00A815CE2C877...
3,3088,53.942,42.170,980.359,8.8820,44.935818,-93.21752,0101000020E6100000758E01D9EB4D57C00A815CE2C877...
4,3088,50.660,40.259,978.572,7.8060,44.935818,-93.21752,0101000020E6100000758E01D9EB4D57C00A815CE2C877...


In [18]:
# trying to visualize on AGOL
#DOES NOT WORK - I think it would work but the credentials don't work

from arcgis.features import FeatureLayer

# Connect to ArcGIS Online using your university credentials
gis = GIS("https://arcgis.com", enterprise_username="and03449", enterprise_password="Spankyandritz2620!", auth_mode="interactive", client_id="UMN", profile="and03449_UMN")

# Set the input table and output feature class
in_table = "historic_latlong"
out_feature_class = "historic_latlong_fc"

# Use XYTableToPoint function to convert the input table to a feature class
arcpy.management.XYTableToPoint(in_table, out_feature_class, "longitude", "latitude")

# Publish the feature class as a feature layer
out_feature_layer = FeatureLayer.from_featureclass(out_feature_class)
published_feature_layer = out_feature_layer.publish()

# Display the feature layer on a map
map = gis.map()
map.add_layer(published_feature_layer)
map

Exception: A general error occurred: local variable 'oauth_info' referenced before assignment

In [27]:
print(historic_latlong.dtypes)

sensor_index1      int64
humidity         float64
temperature      float64
pressure         float64
pm2_5            float64
latitude         float64
longitude        float64
wkt               object
dtype: object


In [28]:
historic_latlong = historic_latlong.drop(columns=['wkt'])

In [41]:
#need to convert df to table before any xytotable
arcpy.da.NumPyArrayToTable(historic_latlong.to_records(), save_path)

C:\Users\tande\AppData\Local\Temp\ipykernel_36192\3510423798.py:1: RuntimeWarning: This version does not support tables with int64 field type.
Downcasting column 'index' type int64 to int32 may result in data loss.
Downcasting column 'sensor_index1' type int64 to int32 may result in data loss.
  arcpy.da.NumPyArrayToTable(historic_latlong.to_records(), save_path)


RuntimeError: invalid table name

In [20]:
# Set the input table and output feature class
in_table = "historic_latlong"
out_feature_class = "historic_latlong_fc"

# Use XYTableToPoint function to convert the input table to a feature class
arcpy.management.XYTableToPoint(in_table, out_feature_class, "longitude", "latitude")

# Convert the feature class to a GeoJSON file
geojson_file = out_feature_class + ".geojson"
arcpy.FeaturesToJSON_conversion(out_feature_class, geojson_file)

# Create a folium map centered on the first feature in the GeoJSON file
map = folium.Map(location=[0, 0], zoom_start=2)
folium.GeoJson(geojson_file).add_to(map)
map.fit_bounds(folium.GeoJson(geojson_file).get_bounds())

# Display the map
map


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset historic_latlong does not exist or is not supported
Failed to execute (XYTableToPoint).


In [35]:
# Set the input table and output feature class
in_table = "historic_latlong"
save_path = os.path.join(arcpy.env.workspace, "historic_latlong_fc")

# Use XYTableToPoint function to convert the input table to a feature class
arcpy.management.XYTableToPoint(in_table, save_path, "longitude", "latitude")


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Table: Dataset historic_latlong does not exist or is not supported
Failed to execute (XYTableToPoint).
